# BTC Monthly Trend Dashboard

This notebook fetches daily Bitcoin data from the CoinGecko API, aggregates it to a monthly frequency and visualizes trading volume, average price and returns.  It is designed as a simple exercise in data analysis and demonstrates how to clean and aggregate time‑series data using pandas and matplotlib.

**Note:** The code checks for a cached CSV file to avoid unnecessary API calls.  If the file does not exist, it will download data from CoinGecko and save it.

In [ ]:
from pathlib import Path
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Create data and image directories
DATA_DIR = Path('data')
IMAGE_DIR = Path('images')
DATA_DIR.mkdir(parents=True, exist_ok=True)
IMAGE_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
def fetch_btc_data(data_path):
    # If the data file exists, load it
    if data_path.exists():
        df = pd.read_csv(data_path, parse_dates=['date'])
        return df

    # Otherwise fetch from CoinGecko
    url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 'max',
        'interval': 'daily'
    }
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    data = response.json()

    # Convert lists of [timestamp, value] to DataFrames
    def series_to_df(series, value_col):
        df_series = pd.DataFrame(series, columns=['timestamp', value_col])
        df_series['date'] = pd.to_datetime(df_series['timestamp'], unit='ms')
        return df_series[['date', value_col]]

    df_price = series_to_df(data.get('prices', []), 'price_usd')
    df_volume = series_to_df(data.get('total_volumes', []), 'volume_usd')
    df = df_price.merge(df_volume, on='date', how='left')
    df = df.sort_values('date').reset_index(drop=True)
    # Save to CSV for caching
    df.to_csv(data_path, index=False)
    return df


In [ ]:
# Path to cached data
data_path = DATA_DIR / 'btc_daily_data.csv'
# Fetch the data
btc_df = fetch_btc_data(data_path)
btc_df = btc_df.sort_values('date').reset_index(drop=True)

# Create a year-month period column
btc_df['year_month'] = btc_df['date'].dt.to_period('M')
monthly_df = (
    btc_df.groupby('year_month').agg(
        monthly_volume=('volume_usd', 'sum'),
        monthly_avg_price=('price_usd', 'mean')
    ).reset_index()
)
monthly_df['month'] = monthly_df['year_month'].dt.to_timestamp()
monthly_df['monthly_return'] = monthly_df['monthly_avg_price'].pct_change()
monthly_df['rolling_return'] = monthly_df['monthly_return'].rolling(window=3).mean()


In [ ]:
# Plot monthly trading volume
plt.figure(figsize=(10, 4))
plt.bar(monthly_df['month'], monthly_df['monthly_volume'], color='steelblue')
plt.title('Monthly Bitcoin Trading Volume (USD)')
plt.xlabel('Month')
plt.ylabel('Total Volume (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
volume_path = IMAGE_DIR / 'monthly_volume.png'
plt.savefig(volume_path)
plt.show()


In [ ]:
# Plot monthly average price
plt.figure(figsize=(10, 4))
plt.plot(monthly_df['month'], monthly_df['monthly_avg_price'], color='green')
plt.title('Monthly Average Bitcoin Price (USD)')
plt.xlabel('Month')
plt.ylabel('Average Price (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
price_path = IMAGE_DIR / 'monthly_average_price.png'
plt.savefig(price_path)
plt.show()


In [ ]:
# Plot monthly returns and 3‑month rolling average
plt.figure(figsize=(10, 4))
plt.bar(monthly_df['month'], monthly_df['monthly_return'], color='orchid', label='Monthly Return')
plt.plot(monthly_df['month'], monthly_df['rolling_return'], color='orange', label='3‑Month Rolling Average')
plt.title('Monthly Returns with 3‑Month Rolling Average')
plt.xlabel('Month')
plt.ylabel('Return')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
returns_path = IMAGE_DIR / 'monthly_returns.png'
plt.savefig(returns_path)
plt.show()
